# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1670856c70>
├── 'a' --> tensor([[-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506]])
└── 'x' --> <FastTreeValue 0x7f1670856910>
    └── 'c' --> tensor([[-2.1743, -0.3953, -1.1315, -1.1594],
                        [-0.1151,  1.8177,  1.5624,  0.3615],
                        [-0.0633,  0.2561,  0.8253, -0.0958]])

In [4]:
t.a

tensor([[-0.8130, -1.0331,  1.2092],
        [ 1.5258, -1.1744,  0.1506]])

In [5]:
%timeit t.a

59.1 ns ± 0.171 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1670856c70>
├── 'a' --> tensor([[-0.6491,  0.5930, -0.7125],
│                   [-0.0746,  0.3927, -0.2570]])
└── 'x' --> <FastTreeValue 0x7f1670856910>
    └── 'c' --> tensor([[-2.1743, -0.3953, -1.1315, -1.1594],
                        [-0.1151,  1.8177,  1.5624,  0.3615],
                        [-0.0633,  0.2561,  0.8253, -0.0958]])

In [7]:
%timeit t.a = new_value

61.7 ns ± 0.0292 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506]]),
    x: Batch(
           c: tensor([[-2.1743, -0.3953, -1.1315, -1.1594],
                      [-0.1151,  1.8177,  1.5624,  0.3615],
                      [-0.0633,  0.2561,  0.8253, -0.0958]]),
       ),
)

In [10]:
b.a

tensor([[-0.8130, -1.0331,  1.2092],
        [ 1.5258, -1.1744,  0.1506]])

In [11]:
%timeit b.a

57.2 ns ± 0.0414 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0322, -0.4092, -0.6401],
               [ 1.7601, -0.5826,  1.6069]]),
    x: Batch(
           c: tensor([[-2.1743, -0.3953, -1.1315, -1.1594],
                      [-0.1151,  1.8177,  1.5624,  0.3615],
                      [-0.0633,  0.2561,  0.8253, -0.0958]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.143 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 31 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 520 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f15cd1ec760>
├── 'a' --> tensor([[[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]],
│           
│                   [[-0.8130, -1.0331,  1.2092],
│                    [ 1.5258, -1.1744,  0.1506]]])
└── 'x' --> <FastTreeValue 0x7f15cd1ec730>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 68.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f16708ccfa0>
├── 'a' --> tensor([[-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506],
│                   [-0.8130, -1.0331,  1.2092],
│                   [ 1.5258, -1.1744,  0.1506]])
└── 'x' --> <FastTreeValue 0x7f15c6c02ac0>
    └── 'c' --> tensor([[-2.1743, -0.3953, -1.1315, -1.1594],
                        [-0.1151,  1.8177,  1.5624,  0.3615],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 72.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.1 µs ± 31.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]],
       
               [[-0.8130, -1.0331,  1.2092],
                [ 1.5258, -1.1744,  0.1506]]]),
    x: Batch(
           c: tensor([[[-2.1743, -0.3953, -1.1315, -1.1594],
                       [-0.1151,  1.8177,  1.5624,  0.3615],
                       [-0.0633,  0.2561,  0.8253, -0.0958]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506],
               [-0.8130, -1.0331,  1.2092],
               [ 1.5258, -1.1744,  0.1506]]),
    x: Batch(
           c: tensor([[-2.1743, -0.3953, -1.1315, -1.1594],
                      [-0.1151,  1.8177,  1.5624,  0.3615],
                      [-0.0633,  0.2561,  0.8253, -0.0958],
                      [-2.1743, -0.3953, -1.1315, -1.1594],
                      [-0.1151,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 530 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
